Материалы

У вас лекционный ноутбук в архиве он с картинками и презентация тоже есть в архиве

Задание

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара 

1. построить свёрточные архитектуры 

2. построить различные архитектуры с RNN

3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)

4. сдлать выводы что получилось лучше

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install stop_words

In [ ]:
!pip install pymorphy2

In [5]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [49]:
def print_res():
  score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
  print('\n')
  print('Test score:', score[0])
  print('Test accuracy:', score[1])

#Предобработка данных

In [8]:
df_train = pd.read_csv("/content/drive/MyDrive/data_less7/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/data_less7/test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/data_less7/val.csv")

In [9]:
df_train.head(2)

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1


In [10]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [11]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [12]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [13]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [14]:
y_train = df_train['class'].values
y_val = df_val['class'].values

#1. Построим сверточную сеть

In [15]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
import tensorflow as tf

In [16]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=128, input_length=training_length, trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 151s 423ms/step - loss: 0.5999 - accuracy: 0.6563 - val_loss: 0.4861 - val_accuracy: 0.7559


In [18]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 32ms/step - loss: 0.5179 - accuracy: 0.7387


Test score: 0.5179295539855957
Test accuracy: 0.7387030124664307


In [75]:
mod = []
acc = []
mod.append('CNN1')
acc.append(0.7387)

для примера еще одну сеть

In [34]:

model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=128, input_length=training_length))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Conv1D(64, 2))
model.add(Activation("relu"))
model.add(Conv1D(64, 1))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
45/45 [==============================] - 2s 34ms/step - loss: 0.5106 - accuracy: 0.7401


Test score: 0.51059490442276
Test accuracy: 0.7400696277618408


In [76]:
mod.append('CNN2')
acc.append(0.7400)

#2. RNN

SimpleRNN

In [36]:
# попробуем запрограммировать простую рекурентную сеть

model = Sequential()
model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 53s 135ms/step - loss: 0.6177 - accuracy: 0.6351 - val_loss: 0.4934 - val_accuracy: 0.7547


In [38]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 19ms/step - loss: 0.5055 - accuracy: 0.7453


Test score: 0.5054731965065002
Test accuracy: 0.7453158497810364


In [77]:
mod.append('SimpleRNN')
acc.append(0.7453)

LSTM

In [39]:
model = Sequential()
model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 86s 249ms/step - loss: 0.6109 - accuracy: 0.6479 - val_loss: 0.4929 - val_accuracy: 0.7533


In [40]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 37ms/step - loss: 0.5092 - accuracy: 0.7440


Test score: 0.5091725587844849
Test accuracy: 0.7439932823181152


In [78]:
mod.append('LSTM')
acc.append(0.7439)

GRU 


In [41]:
model = Sequential()
model.add(Embedding(input_dim=word_count,input_length=training_length,output_dim=30,trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train,batch_size=512,epochs=10,verbose=1,validation_split=0.1,callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 75s 219ms/step - loss: 0.6096 - accuracy: 0.6469 - val_loss: 0.4949 - val_accuracy: 0.7542


In [42]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 32ms/step - loss: 0.5078 - accuracy: 0.7430


Test score: 0.5077802538871765
Test accuracy: 0.7430233955383301


In [79]:
mod.append('GRU')
acc.append(0.7430)

#3 совместные архитектуры CNN -> RNN 

CNN-SimpleRNN

In [59]:
model = Sequential()
model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])
print_res()

Epoch 1/10
45/45 [==============================] - 1s 27ms/step - loss: 0.5180 - accuracy: 0.7443


Test score: 0.517991840839386
Test accuracy: 0.7442578077316284


In [80]:
mod.append('CNN-SimpleRNN')
acc.append(0.7442)

CNN - LSTM

In [60]:
model = Sequential()
model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])
print_res()

Epoch 1/10
45/45 [==============================] - 2s 54ms/step - loss: 0.5118 - accuracy: 0.7442


Test score: 0.5118025541305542
Test accuracy: 0.7441696524620056


In [81]:
mod.append('CNN - LSTM')
acc.append(0.7441)

CNN-GRU

In [61]:
model = Sequential()
model.add(Embedding(input_dim=word_count,input_length=training_length,output_dim=30,trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train,batch_size=512,epochs=10,verbose=1,validation_split=0.1,callbacks=[early_stopping])
print_res()

Epoch 1/10
45/45 [==============================] - 2s 44ms/step - loss: 0.5072 - accuracy: 0.7468


Test score: 0.5072427988052368
Test accuracy: 0.7468147873878479


In [82]:
mod.append('CNN-GRU')
acc.append(0.7468)

#4 совместные архитектуры RNN ->  CNN

SimpleRNN-CNN

In [68]:
model = Sequential()
model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30, trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])
print_res()

Epoch 1/10
45/45 [==============================] - 1s 32ms/step - loss: 0.6648 - accuracy: 0.5514


Test score: 0.6648169755935669
Test accuracy: 0.5514278411865234


In [83]:
mod.append('SimpleRNN-CNN')
acc.append(0.5514)

LSTM - CNN 

In [70]:
model = Sequential()
model.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30))
model.add(LSTM(64, recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])
print_res()

Epoch 1/10
45/45 [==============================] - 2s 47ms/step - loss: 0.6655 - accuracy: 0.5513


Test score: 0.6654847264289856
Test accuracy: 0.5512726902961731


In [84]:
mod.append('LSTM - CNN ')
acc.append(0.5512)

GRU-CNN

In [71]:
model = Sequential()
model.add(Embedding(input_dim=word_count,input_length=training_length,output_dim=30,trainable=True, mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(64, recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train,batch_size=512,epochs=10,verbose=1,validation_split=0.1,callbacks=[early_stopping])
print_res()

Epoch 1/10
45/45 [==============================] - 2s 51ms/step - loss: 0.6643 - accuracy: 0.5463


Test score: 0.6643022894859314
Test accuracy: 0.5462594032287598


In [85]:
mod.append('GRU-CNN')
acc.append(0.5462)

#5 Результаты

In [86]:
for i in range(len(mod)):
  print(f'модель {mod[i]} accuracy {acc[i]}')

модель CNN1 accuracy 0.7387
модель CNN2 accuracy 0.74
модель SimpleRNN accuracy 0.7453
модель LSTM accuracy 0.7439
модель GRU accuracy 0.743
модель CNN-SimpleRNN accuracy 0.7442
модель CNN - LSTM accuracy 0.7441
модель CNN-GRU accuracy 0.7468
модель SimpleRNN-CNN accuracy 0.5514
модель LSTM - CNN  accuracy 0.5512
модель GRU-CNN accuracy 0.5462


Лучший результат получился у сети модель CNN-GRU accuracy 0.7468

в случае, где сначала идет рекуррентный слой, а затем сверточный - результаты очень плохие

все сети содержат одинаковые слои